In [1]:
import matplotlib.pyplot as plt
import arviz as az

from functions import hierarchical_reg_one_protein, get_biomolecular_data
from difference_plot import plot_cs_differences, plot_reference_curves

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Users\User\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
az.style.use("arviz-darkgrid")

In [4]:
protein = '1D3Z'
bmrb_code = '6457'
samples = 2000

In [5]:
dataframe = get_biomolecular_data(protein, bmrb_code)

C:\Users\User\Documents\Agustina\Hierarchical-Bayes-NMR-Validation\_cheshift.py:662: RuntimeWarning: Mean of empty slice
  cs_theo_ave = np.nanmean(cs_theo_array, axis=0)


In [7]:
dataframe.head()

,pdb_code,res,bmrb_code,ca_exp,ca_teo
1,1D3Z,GLN,6457,55.08,55.062
2,1D3Z,ILE,6457,59.57,57.567
3,1D3Z,PHE,6457,55.21,55.370
4,1D3Z,VAL,6457,60.62,60.077
5,1D3Z,LYS,6457,54.52,51.846


In [8]:
(pps, 
index, 
categories, 
dataframe, 
trace_che, model) = hierarchical_reg_one_protein(dataframe)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


FloatingPointError: divide by zero encountered in log

In [6]:
with plt.style.context('default'):
    ax, dataframe, perc_dict = plot_cs_differences(protein, 
                                                    save=True, 
                                                    pymol_session=True, 
                                                    bmrb_code=bmrb_code)

C:\Users\User\Documents\Agustina\Hierarchical-Bayes-NMR-Validation\_cheshift.py:662: RuntimeWarning: Mean of empty slice
  cs_theo_ave = np.nanmean(cs_theo_array, axis=0)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


FloatingPointError: divide by zero encountered in log

# LOO

In [ ]:
mean_exp = 56.70
std_exp = 4.90

In [ ]:
idata = az.from_pymc3(trace_che, posterior_predictive={"cheshift":pps})
idata.observed_data['cheshift'] = idata.observed_data['cheshift'] * std_exp + mean_exp

In [ ]:
az.plot_loo_pit(idata, y="cheshift", legend=False)

# $\hat \kappa $ parameter

In [ ]:
loo = az.loo(trace_che, pointwise=True)
az.plot_khat(loo_d3z, hlines_kwargs={"alpha":0})

# ELPD

In [ ]:
_, ax = plt.subplots(figsize=(10,4))
waic = az.waic(trace_che).waic_i.values
ax.scatter(range(len(waic)), waic)
ax.axhline(0, 0, len(waic), linestyle='dashed') 